In [ ]:
from math import sqrt
from numpy import concatenate
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from fbprophet import Prophet
from google.colab import files
from sklearn.metrics import mean_absolute_error as mae


# Reading the data and basic sanity checks

In [ ]:
#Reading the data from text file
meter_data = pd.read_table('/household_power_consumption.txt', sep= ';',parse_dates=[['Date','Time']],infer_datetime_format=True,low_memory=False,na_values=['nan','?'])
meter_data.set_index('Date_Time', inplace = True)

#Reading the weather data from csv (web-scraped file)
weather_data= pd.read_csv('/Weather.csv')
weather_data['Date']= pd.to_datetime(weather_data['Date'])
weather_data.set_index('Date', inplace = True)

meter_data.interpolate(method='linear',limit_direction='forward', axis=0, inplace=True)

weather_data.drop('Precipitation', axis = 1, inplace = True)

# Resampling at daily,weekly and monthly levels

## For Univariate case

In [ ]:
#Resampling meter data at daily, weekly and monthly levels
meter_data_daily = pd.DataFrame(meter_data['Global_active_power'].resample('D').sum())
meter_data_weekly = pd.DataFrame(meter_data['Global_active_power'].resample('W').sum())
meter_data_monthly = pd.DataFrame(meter_data['Global_active_power'].resample('M').sum())

# Creating a column with the power consumption values of the previous instance 

# for days
meter_data_daily_prev = pd.DataFrame(meter_data['Global_active_power'].resample('D').sum())
meter_data_daily_prev.columns = ['Previous day Power']
meter_data_daily_prev = meter_data_daily_prev.shift(1)

# for weeks
meter_data_weekly_prev = pd.DataFrame(meter_data['Global_active_power'].resample('W').sum())
meter_data_weekly_prev.columns = ['Previous week Power']
meter_data_weekly_prev = meter_data_weekly_prev.shift(1)

# for months
meter_data_monthly_prev = pd.DataFrame(meter_data['Global_active_power'].resample('M').sum())
meter_data_monthly_prev.columns = ['Previous month Power']
meter_data_monthly_prev = meter_data_monthly_prev.shift(1)


#Merge the response column from meter data with weather data for daily, weekly and monthly data

data_response_daily = meter_data_daily_prev.merge(meter_data_daily, left_index = True,right_index = True)

data_response_weekly = meter_data_weekly_prev.merge(meter_data_weekly, left_index = True,right_index = True)

data_response_monthly = meter_data_monthly_prev.merge(meter_data_monthly, left_index = True,right_index = True)


#Sanity check
data_response_daily 
#data_response_weekly
# data_response_monthly

,Previous day Power,Global_active_power
Date_Time,,
2006-12-16,NaN,1209.176
2006-12-17,1209.176,3390.460
2006-12-18,3390.460,2203.826
2006-12-19,2203.826,1666.194
2006-12-20,1666.194,2225.748
...,...,...
2010-11-22,900.910,2041.536
2010-11-23,2041.536,1577.536
2010-11-24,1577.536,1796.248


## For Multivariate case

In [ ]:
#Resampling meter data at daily, weekly and monthly levels
meter_data_daily = pd.DataFrame(meter_data['Global_active_power'].resample('D').sum())
meter_data_weekly = pd.DataFrame(meter_data['Global_active_power'].resample('W').sum())
meter_data_monthly = pd.DataFrame(meter_data['Global_active_power'].resample('M').sum())

# Creating a column with the power consumption values of the previous instance 

# for days
meter_data_daily_prev = pd.DataFrame(meter_data['Global_active_power'].resample('D').sum())
meter_data_daily_prev.columns = ['Previous day Power']
meter_data_daily_prev = meter_data_daily_prev.shift(1)

# for weeks
meter_data_weekly_prev = pd.DataFrame(meter_data['Global_active_power'].resample('W').sum())
meter_data_weekly_prev.columns = ['Previous week Power']
meter_data_weekly_prev = meter_data_weekly_prev.shift(1)

# for months
meter_data_monthly_prev = pd.DataFrame(meter_data['Global_active_power'].resample('M').sum())
meter_data_monthly_prev.columns = ['Previous month Power']
meter_data_monthly_prev = meter_data_monthly_prev.shift(1)


#Resampling meter data at daily, weekly and monthly levels
weather_data_daily  = weather_data.copy()
weather_data_weekly  = weather_data.resample('W').mean()
weather_data_monthly  = weather_data.resample('M').mean()



#Merge the response column from meter data with weather data for daily, weekly and monthly data

weather_response_daily = weather_data_daily.shift(1).merge(meter_data_daily_prev, left_index = True,right_index = True)
data_response_daily = weather_response_daily.merge(meter_data_daily, left_index = True,right_index = True)

weather_response_weekly = weather_data_weekly.merge(meter_data_weekly_prev, left_index = True,right_index = True)
data_response_weekly = weather_response_weekly.merge(meter_data_weekly, left_index = True,right_index = True)

weather_response_monthly = weather_data_monthly.merge(meter_data_monthly_prev, left_index = True,right_index = True)
data_response_monthly = weather_response_monthly.merge(meter_data_monthly, left_index = True,right_index = True)

data_response_daily.drop(['Dew Point','Pressure'],axis = 1, inplace = True)
data_response_weekly.drop(['Dew Point','Pressure'],axis = 1, inplace = True)
data_response_monthly.drop(['Dew Point','Pressure'],axis = 1, inplace = True)

#Sanity check
data_response_daily 
# weather_response_weekly
# weather_response_monthly

,Temperature,Humidity,Wind Speed,Previous day Power,Global_active_power
2006-12-16,28.9,88.3,5.3,NaN,1209.176
2006-12-17,42.2,90.0,8.5,1209.176,3390.460
2006-12-18,25.1,93.6,3.4,3390.460,2203.826
2006-12-19,38.1,91.6,5.0,2203.826,1666.194
2006-12-20,36.0,82.3,11.5,1666.194,2225.748
...,...,...,...,...,...
2010-11-22,43.0,94.6,8.3,900.910,2041.536
2010-11-23,39.8,87.1,7.2,2041.536,1577.536
2010-11-24,38.6,89.5,5.3,1577.536,1796.248
2010-11-25,40.3,89.2,6.8,1796.248,1431.164


# Model Training and prediction


## For days

In [ ]:
dataset = weather_response_daily
train,test = dataset.values[:-331],dataset.values[-331:]
#dataset.iloc[1:-331,:-1]
#dataset

### For 1 layer

In [ ]:
def Multivariate_LSTM(X):
  #Splitting the data into train and test sets. Since we are dealing with time-series data we cannot split it into randomized split. It must be a sequential split.
  #Here we are using first 75% data for training and 25% data for testing
  #X_train = X.iloc[:int(X.shape[0]*0.80),:]
  #X_test = X.iloc[int(X.shape[0]*0.80):,:]
  #y_train = X.iloc[:int(X.shape[0]*0.80),-1]
  #y_test = X.iloc[int(X.shape[0]*0.80):,-1]

  X_train = X.iloc[1:-331,:-1]
  X_test = X.iloc[-331:,:-1]
  y_train = X.iloc[1:-331,-1]
  y_test = X.iloc[-331:,-1]

  y_train = y_train.values.reshape(len(y_train),1)
  y_test = y_test.values.reshape(len(y_test),1)

  ####Scaling the data after train-test split
  sc_X = StandardScaler()
  sc_y = StandardScaler()
  X_train_normalized = sc_X.fit_transform(X_train)
  X_test_normalized = sc_X.transform(X_test)
  y_train_normalized = sc_y.fit_transform(y_train)
  y_test_normalized = sc_y.transform(y_test)
  
  # reshape input to be 3D [samples, timesteps, features]
  train_X = X_train_normalized.reshape((X_train_normalized.shape[0], 1, X_train_normalized.shape[1]))
  test_X = X_test_normalized.reshape((X_test_normalized.shape[0], 1, X_test_normalized.shape[1]))

  n_input = 1
  n_features = X_train_normalized.shape[1]

  # define model
  model = Sequential()
  model.add(LSTM(70, activation='relu', input_shape=(n_input, n_features)))
  model.add(Dropout(0.2))
  model.add(Dense(1))
  opt = keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer=opt, loss='mse')
  # fit model
  
  history = model.fit(train_X, y_train_normalized, epochs=100, batch_size=2, validation_data=(test_X, y_test_normalized), 
                      verbose=1, shuffle=False)

  model.summary()

  # plot history
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show()

  # make a prediction
  yhat = model.predict(test_X)
  #print(yhat.shape)
  test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
  
  # invert scaling for forecast
  inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
  inv_yhat = sc_y.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  #print(inv_yhat)
  
  # invert scaling for actual
  y_test_normalized = y_test_normalized.reshape((len(y_test_normalized), 1))
  inv_y = concatenate((y_test_normalized, test_X[:, 1:]), axis=1)
  inv_y = sc_y.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  #print(inv_y)

  
  #plt.plot(inv_y, 'go', linewidth=4, label='Actual Value')
  #plt.plot(inv_yhat,'rx--', linewidth=2, label='Predicted Value')
  #plt.legend()
  #plt.savefig("abc.png")
  #files.download("abc.png")
  #plt.show()

  fig2 = plt.figure(figsize=(8, 6)) # The size of the figure is specified as (width, height) in inches

  # lines:
  l1 = fig2.add_subplot(111).plot(inv_y, 'co-')
  l2 = fig2.add_subplot(111).plot(inv_yhat,'rx--')
  fig2.add_subplot(111).legend(['Actual', 'Predicted'], loc=0)


  # axes:
  fig2.add_subplot(111).grid(True)

  # labels:
  fig2.add_subplot(111).set_xlabel(r"Days", fontsize=15)
  fig2.add_subplot(111).set_ylabel(r"Global_active_power (in Kilowatts)", fontsize=15)
  fig2.add_subplot(111).set_title(r"Actual vs. Predicted for daily case", fontsize=15)

  # saving:
  fig2.savefig("lstm_multivariate_days.png", dpi=150)
  files.download("lstm_multivariate_days.png")
  fig2.show()



  print(inv_y.shape)

  #print(inv_yhat.shape)
  #print(inv_y[:])

  res = np.vstack((inv_y, inv_yhat))
  res = res.transpose()
  DF = pd.DataFrame(res)
  # save the dataframe as a csv file
  DF.to_csv("results.csv")

  # calculate RMSE
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
  mape = np.mean(np.abs((inv_y.flatten() - inv_yhat.flatten()) /inv_y.flatten())) * 100
  print(f'Mean absolute percentage error is: {mape}')
  mae_error = mae(inv_y.flatten(),inv_yhat.flatten())
  print(f'Mean Absolute Error is: {mae_error} ')


In [ ]:
X = data_response_daily.copy()
Multivariate_LSTM(X)

### For multi-layer

In [ ]:
def Multivariate_LSTM_2(X):
  #Splitting the data into train and test sets. Since we are dealing with time-series data we cannot split it into randomized split. It must be a sequential split.
  #Here we are using first 75% data for training and 25% data for testing
  #X_train = X.iloc[:int(X.shape[0]*0.80),:]
  #X_test = X.iloc[int(X.shape[0]*0.80):,:]
  #y_train = X.iloc[:int(X.shape[0]*0.80),-1]
  #y_test = X.iloc[int(X.shape[0]*0.80):,-1]

  X_train = X.iloc[1:-331,:-1]
  X_test = X.iloc[-331:,:-1]
  y_train = X.iloc[1:-331,-1]
  y_test = X.iloc[-331:,-1]

  y_train = y_train.values.reshape(len(y_train),1)
  y_test = y_test.values.reshape(len(y_test),1)

  ####Scaling the data after train-test split
  sc_X = StandardScaler()
  sc_y = StandardScaler()
  X_train_normalized = sc_X.fit_transform(X_train)
  X_test_normalized = sc_X.transform(X_test)
  y_train_normalized = sc_y.fit_transform(y_train)
  y_test_normalized = sc_y.transform(y_test)

  # reshape input to be 3D [samples, timesteps, features]
  train_X = X_train_normalized.reshape((X_train_normalized.shape[0], 1, X_train_normalized.shape[1]))
  test_X = X_test_normalized.reshape((X_test_normalized.shape[0], 1, X_test_normalized.shape[1]))

  n_input = 1
  n_features = X_train_normalized.shape[1]

  # define model
  model = Sequential()
  model.add(LSTM(70, activation='relu', return_sequences = True, input_shape=(n_input, n_features)))
  model.add(Dropout(0.2))
  model.add(LSTM(30, activation='relu',return_sequences = True))
  model.add(Dropout(0.1))
  # can add more layers if required
  model.add(Dense(1))
  opt = keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer=opt, loss='mse')
  # fit model
  history = model.fit(train_X, y_train_normalized, epochs=150, batch_size=10, validation_data=(test_X, y_test_normalized), 
                      verbose=1, shuffle=False)

  model.summary()

  # plot history
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show()

  # make a prediction
  yhat = model.predict(test_X)
  yhat.shape
  test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
  
  # invert scaling for forecast
  yhat = yhat.reshape((yhat.shape[0], yhat.shape[2]))
  inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
  inv_yhat = sc_y.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  #print(inv_yhat)
  
  # invert scaling for actual
  y_test_normalized = y_test_normalized.reshape((len(y_test_normalized), 1))
  inv_y = concatenate((y_test_normalized, test_X[:, 1:]), axis=1)
  inv_y = sc_y.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  #print(inv_y)


  #plt.plot(inv_y, 'go', linewidth=4, label='Actual Value')
  #plt.plot(inv_yhat,'rx--', linewidth=2, label='Predicted Value')
  #plt.legend()
  #plt.savefig("abc.png")
  #files.download("abc.png")
  #plt.show()

  fig2 = plt.figure(figsize=(8, 6)) # The size of the figure is specified as (width, height) in inches

  # lines:
  l1 = fig2.add_subplot(111).plot(inv_y, 'co-')
  l2 = fig2.add_subplot(111).plot(inv_yhat,'rx--')
  fig2.add_subplot(111).legend(['Actual', 'Predicted'], loc=0)


  # axes:
  fig2.add_subplot(111).grid(True)

  # labels:
  fig2.add_subplot(111).set_xlabel(r"Days", fontsize=15)
  fig2.add_subplot(111).set_ylabel(r"Global_active_power (in Kilowatts)", fontsize=15)
  fig2.add_subplot(111).set_title(r"Actual vs. Predicted for daily case", fontsize=15)

  # saving:
  fig2.savefig("lstm_multivariate_days.png", dpi=150)
  files.download("lstm_multivariate_days.png")
  fig2.show()



  #print(inv_y.shape)

  #print(inv_yhat.shape)
  #print(inv_y[:])

  # calculate RMSE
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
  mape = np.mean(np.abs((inv_y.flatten() - inv_yhat.flatten()) /inv_y.flatten())) * 100
  print(f'Mean absolute percentage error is: {mape}')
  mae_error = mae(inv_y.flatten(),inv_yhat.flatten())
  print(f'Mean Absolute Error is: {mae_error} ')


In [ ]:
X = data_response_daily.copy()
Multivariate_LSTM_2(X)

### For timesteps

In [ ]:
def timesteps(x_train,y_train,lag):
  X = []
  Y = []
  num_samples = x_train.shape[0]

  for i in range(lag, num_samples):

    temp = x_train[i-lag:i, :]
    #X.append(temp.reshape(1,temp.shape[0],temp.shape[1]))
    X.append(temp)
    Y.append(y_train[i])

  X,Y = np.array(X), np.array(Y)
  
  return X,Y


In [ ]:
def Multivariate_LSTM_timesteps(X,lag):
  #Splitting the data into train and test sets. Since we are dealing with time-series data we cannot split it into randomized split. It must be a sequential split.
  #Here we are using first 75% data for training and 25% data for testing
  #X_train = X.iloc[:int(X.shape[0]*0.80),:]
  #X_test = X.iloc[int(X.shape[0]*0.80):,:]
  #y_train = X.iloc[:int(X.shape[0]*0.80),-1]
  #y_test = X.iloc[int(X.shape[0]*0.80):,-1]

  X_train = X.iloc[1:-331,:-1]
  X_test = X.iloc[-331:,:-1]
  y_train = X.iloc[1:-331,-1]
  y_test = X.iloc[-331:,-1]

  y_train = y_train.values.reshape(len(y_train),1)
  y_test = y_test.values.reshape(len(y_test),1)

  ####Scaling the data after train-test split
  sc_X = StandardScaler()
  sc_y = StandardScaler()
  X_train_normalized = sc_X.fit_transform(X_train)
  X_test_normalized = sc_X.transform(X_test)
  y_train_normalized = sc_y.fit_transform(y_train)
  y_test_normalized = sc_y.transform(y_test)

  # Creating data in timesteps
  Xt_train,Yt_train = timesteps(X_train_normalized,y_train_normalized,lag)
  Xt_test,Yt_test = timesteps(X_test_normalized,y_test_normalized,lag)
  
  # reshape input to be 3D [samples, timesteps, features]
  train_X = Xt_train
  test_X = Xt_test

  n_input = lag
  n_features = train_X.shape[2]

  # define model
  model = Sequential()
  model.add(LSTM(100, activation='relu', return_sequences = True, input_shape=(n_input, n_features)))
  model.add(Dropout(0.2))
  model.add(LSTM(35, activation='relu',return_sequences = True))
  model.add(Dropout(0.1))
  model.add(Dense(1))
  opt = keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer=opt, loss='mse')
  # fit model
  history = model.fit(train_X, Yt_train, epochs=150, batch_size=40, validation_data=(test_X, Yt_test), 
                     verbose=1, shuffle=False)

  model.summary()

  # plot history
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show()

  # make a prediction
  yhat = model.predict(test_X)
  #print(yhat.shape)
  test_X = test_X[:,-1,:].reshape((test_X.shape[0], test_X.shape[2]))

  # invert scaling for forecast
  yhat = yhat[:,-1,:].reshape((yhat.shape[0], yhat.shape[2]))
  inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
  inv_yhat = sc_y.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  #print(inv_yhat)

  # invert scaling for actual
  y_test_normalized = y_test_normalized.reshape((len(y_test_normalized), 1))
  inv_y = concatenate((Yt_test, test_X[:, 1:]), axis=1)
  inv_y = sc_y.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  #print(inv_y)


  #plt.plot(inv_y, 'go', linewidth=4, label='Actual Value')
  #plt.plot(inv_yhat,'rx--', linewidth=2, label='Predicted Value')
  #plt.legend()
  #plt.savefig("abc.png")
  #files.download("abc.png")
  #plt.show()

  fig2 = plt.figure(figsize=(8, 6)) # The size of the figure is specified as (width, height) in inches

  # lines:
  l1 = fig2.add_subplot(111).plot(inv_y, 'co-')
  l2 = fig2.add_subplot(111).plot(inv_yhat,'rx--')
  fig2.add_subplot(111).legend(['Actual', 'Predicted'], loc=0)


  # axes:
  fig2.add_subplot(111).grid(True)

  # labels:
  fig2.add_subplot(111).set_xlabel(r"Days", fontsize=15)
  fig2.add_subplot(111).set_ylabel(r"Global_active_power (in Kilowatts)", fontsize=15)
  fig2.add_subplot(111).set_title(r"Actual vs. Predicted for daily case", fontsize=15)

  # saving:
  fig2.savefig("lstm_multivariate_days.png", dpi=150)
  files.download("lstm_multivariate_days.png")
  fig2.show()



  #print(inv_y.shape)

  #print(inv_yhat.shape)
  #print(inv_y[:])

  # calculate RMSE
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
  mape = np.mean(np.abs((inv_y.flatten() - inv_yhat.flatten()) /inv_y.flatten())) * 100
  print(f'Mean absolute percentage error is: {mape}')
  mae_error = mae(inv_y.flatten(),inv_yhat.flatten())
  print(f'Mean Absolute Error is: {mae_error} ')


In [ ]:
X = data_response_daily.copy()
Multivariate_LSTM_timesteps(X,10)

## For weeks

### For 1 layer


In [ ]:
dataset = weather_response_weekly
#train,test = dataset.values[:-48],dataset.values[-48:]
#dataset[1:-331]
dataset[1:-48]
dataset.iloc[1:-48]

In [ ]:
def Multivariate_LSTM(X):
  #Splitting the data into train and test sets. Since we are dealing with time-series data we cannot split it into randomized split. It must be a sequential split.
  #Here we are using first 75% data for training and 25% data for testing

  X_train = X.iloc[1:-48,:-1]
  X_test = X.iloc[-48:,:-1]
  y_train = X.iloc[1:-48,-1]
  y_test = X.iloc[-48:,-1]

  y_train = y_train.values.reshape(len(y_train),1)
  y_test = y_test.values.reshape(len(y_test),1)

  ####Scaling the data after train-test split
  sc_X = StandardScaler()
  sc_y = StandardScaler()
  X_train_normalized = sc_X.fit_transform(X_train)
  X_test_normalized = sc_X.transform(X_test)
  y_train_normalized = sc_y.fit_transform(y_train)
  y_test_normalized = sc_y.transform(y_test)
  
  # reshape input to be 3D [samples, timesteps, features]
  train_X = X_train_normalized.reshape((X_train_normalized.shape[0], 1, X_train_normalized.shape[1]))
  test_X = X_test_normalized.reshape((X_test_normalized.shape[0], 1, X_test_normalized.shape[1]))

  n_input = 1
  n_features = X_train_normalized.shape[1]

  # define model
  model = Sequential()
  model.add(LSTM(60, activation='relu', input_shape=(n_input, n_features)))
  model.add(Dropout(0.2))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mse')
  # fit model
  history = model.fit(train_X, y_train_normalized, epochs=100, batch_size= 5, validation_data=(test_X, y_test_normalized), 
                     verbose=1, shuffle=False)

  model.summary()

  # plot history
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show()

  # make a prediction
  yhat = model.predict(test_X)
  test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
  
  # invert scaling for forecast
  inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
  inv_yhat = sc_y.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  #print(inv_yhat)
  
  # invert scaling for actual
  y_test_normalized = y_test_normalized.reshape((len(y_test_normalized), 1))
  inv_y = concatenate((y_test_normalized, test_X[:, 1:]), axis=1)
  inv_y = sc_y.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  #print(inv_y)

  
  #plt.plot(inv_y, 'go', linewidth=4, label='Actual Value')
  #plt.plot(inv_yhat,'rx--', linewidth=2, label='Predicted Value')
  #plt.legend()
  #plt.savefig("abc.png")
  #files.download("abc.png")
  #plt.show()

  fig2 = plt.figure(figsize=(8, 6)) # The size of the figure is specified as (width, height) in inches

  # lines:
  l1 = fig2.add_subplot(111).plot(inv_y, 'co-')
  l2 = fig2.add_subplot(111).plot(inv_yhat,'rx--')
  fig2.add_subplot(111).legend(['Actual', 'Predicted'], loc=0)


  # axes:
  fig2.add_subplot(111).grid(True)

  # labels:
  fig2.add_subplot(111).set_xlabel(r"Weeks", fontsize=15)
  fig2.add_subplot(111).set_ylabel(r"Global_active_power (in Kilowatts)", fontsize=15)
  fig2.add_subplot(111).set_title(r"Actual vs. Predicted for Weekly case", fontsize=15)

  # saving:
  fig2.savefig("lstm_multivariate_weeks.png", dpi=150)
  files.download("lstm_multivariate_weeks.png")
  fig2.show()



  #print(inv_y.shape)

  #print(inv_yhat.shape)
  #print(inv_y[:])

  res = np.vstack((inv_y, inv_yhat))
  res = res.transpose()
  DF = pd.DataFrame(res)
  # save the dataframe as a csv file
  DF.to_csv("results.csv")

  # calculate RMSE
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
  mape = np.mean(np.abs((inv_y.flatten() - inv_yhat.flatten()) /inv_y.flatten())) * 100
  print(f'Mean absolute percentage error is: {mape}')
  mae_error = mae(inv_y.flatten(),inv_yhat.flatten())
  print(f'Mean Absolute Error is: {mae_error} ')


In [ ]:
X = data_response_weekly.copy()
Multivariate_LSTM(X)

### For multi-layer

In [ ]:
def Multivariate_LSTM_2(X):
  #Splitting the data into train and test sets. Since we are dealing with time-series data we cannot split it into randomized split. It must be a sequential split.
  #Here we are using first 75% data for training and 25% data for testing
  #X_train = X.iloc[:int(X.shape[0]*0.80),:]
  #X_test = X.iloc[int(X.shape[0]*0.80):,:]
  #y_train = X.iloc[:int(X.shape[0]*0.80),-1]
  #y_test = X.iloc[int(X.shape[0]*0.80):,-1]

  X_train = X.iloc[1:-48,:-1]
  X_test = X.iloc[-48:,:-1]
  y_train = X.iloc[1:-48,-1]
  y_test = X.iloc[-48:,-1]

  y_train = y_train.values.reshape(len(y_train),1)
  y_test = y_test.values.reshape(len(y_test),1)

  ####Scaling the data after train-test split
  sc_X = StandardScaler()
  sc_y = StandardScaler()
  X_train_normalized = sc_X.fit_transform(X_train)
  X_test_normalized = sc_X.transform(X_test)
  y_train_normalized = sc_y.fit_transform(y_train)
  y_test_normalized = sc_y.transform(y_test)

  # reshape input to be 3D [samples, timesteps, features]
  train_X = X_train_normalized.reshape((X_train_normalized.shape[0], 1, X_train_normalized.shape[1]))
  test_X = X_test_normalized.reshape((X_test_normalized.shape[0], 1, X_test_normalized.shape[1]))

  n_input = 1
  n_features = X_train_normalized.shape[1]

  # define model
  model = Sequential()
  model.add(LSTM(70, activation='relu', return_sequences = True, input_shape=(n_input, n_features)))
  model.add(Dropout(0.3))
  model.add(LSTM(30, activation='relu',return_sequences = True))
  model.add(Dropout(0.1))
  # can add more layers if required
  model.add(Dense(1))
  opt = keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer=opt, loss='mse')
  # fit model
  history = model.fit(train_X, y_train_normalized, epochs=150, batch_size=3, validation_data=(test_X, y_test_normalized), 
                      verbose=1, shuffle=False)

  model.summary()

  # plot history
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show()

  # make a prediction
  yhat = model.predict(test_X)
  yhat.shape
  test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
  
  # invert scaling for forecast
  yhat = yhat.reshape((yhat.shape[0], yhat.shape[2]))
  inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
  inv_yhat = sc_y.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  #print(inv_yhat)
  
  # invert scaling for actual
  y_test_normalized = y_test_normalized.reshape((len(y_test_normalized), 1))
  inv_y = concatenate((y_test_normalized, test_X[:, 1:]), axis=1)
  inv_y = sc_y.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  #print(inv_y)


  #plt.plot(inv_y, 'go', linewidth=4, label='Actual Value')
  #plt.plot(inv_yhat,'rx--', linewidth=2, label='Predicted Value')
  #plt.legend()
  #plt.savefig("abc.png")
  #files.download("abc.png")
  #plt.show()

  fig2 = plt.figure(figsize=(8, 6)) # The size of the figure is specified as (width, height) in inches

  # lines:
  l1 = fig2.add_subplot(111).plot(inv_y, 'co-')
  l2 = fig2.add_subplot(111).plot(inv_yhat,'rx--')
  fig2.add_subplot(111).legend(['Actual', 'Predicted'], loc=0)


  # axes:
  fig2.add_subplot(111).grid(True)

  # labels:
  fig2.add_subplot(111).set_xlabel(r"Days", fontsize=15)
  fig2.add_subplot(111).set_ylabel(r"Global_active_power (in Kilowatts)", fontsize=15)
  fig2.add_subplot(111).set_title(r"Actual vs. Predicted for daily case", fontsize=15)

  # saving:
  fig2.savefig("lstm_multivariate_days.png", dpi=150)
  files.download("lstm_multivariate_days.png")
  fig2.show()



  #print(inv_y.shape)

  #print(inv_yhat.shape)
  #print(inv_y[:])

  # calculate RMSE
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
  mape = np.mean(np.abs((inv_y.flatten() - inv_yhat.flatten()) /inv_y.flatten())) * 100
  print(f'Mean absolute percentage error is: {mape}')
  mae_error = mae(inv_y.flatten(),inv_yhat.flatten())
  print(f'Mean Absolute Error is: {mae_error} ')

In [ ]:
X = data_response_weekly.copy()
Multivariate_LSTM_2(X)

### For timesteps

In [ ]:
def timesteps(x_train,y_train,lag):
  X = []
  Y = []
  num_samples = x_train.shape[0]

  for i in range(lag, num_samples):

    temp = x_train[i-lag:i, :]
    #X.append(temp.reshape(1,temp.shape[0],temp.shape[1]))
    X.append(temp)
    Y.append(y_train[i])

  X,Y = np.array(X), np.array(Y)
  
  return X,Y

In [ ]:
def Multivariate_LSTM_timesteps(X,lag):
  #Splitting the data into train and test sets. Since we are dealing with time-series data we cannot split it into randomized split. It must be a sequential split.
  #Here we are using first 75% data for training and 25% data for testing
  #X_train = X.iloc[:int(X.shape[0]*0.80),:]
  #X_test = X.iloc[int(X.shape[0]*0.80):,:]
  #y_train = X.iloc[:int(X.shape[0]*0.80),-1]
  #y_test = X.iloc[int(X.shape[0]*0.80):,-1]

  X_train = X.iloc[1:-48,:-1]
  X_test = X.iloc[-48:,:-1]
  y_train = X.iloc[1:-48,-1]
  y_test = X.iloc[-48:,-1]

  y_train = y_train.values.reshape(len(y_train),1)
  y_test = y_test.values.reshape(len(y_test),1)

  ####Scaling the data after train-test split
  sc_X = StandardScaler()
  sc_y = StandardScaler()
  X_train_normalized = sc_X.fit_transform(X_train)
  X_test_normalized = sc_X.transform(X_test)
  y_train_normalized = sc_y.fit_transform(y_train)
  y_test_normalized = sc_y.transform(y_test)

  # Creating data in timesteps
  Xt_train,Yt_train = timesteps(X_train_normalized,y_train_normalized,lag)
  Xt_test,Yt_test = timesteps(X_test_normalized,y_test_normalized,lag)
  
  # reshape input to be 3D [samples, timesteps, features]
  train_X = Xt_train
  test_X = Xt_test

  n_input = lag
  n_features = train_X.shape[2]

  # define model
  model = Sequential()
  model.add(LSTM(100, activation='relu', return_sequences = True, input_shape=(n_input, n_features)))
  model.add(Dropout(0.2))
  model.add(LSTM(35, activation='relu',return_sequences = True))
  model.add(Dropout(0.1))
  model.add(Dense(1))
  opt = keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer=opt, loss='mse')
  # fit model
  history = model.fit(train_X, Yt_train, epochs=70, batch_size=3, validation_data=(test_X, Yt_test), 
                     verbose=1, shuffle=False)

  model.summary()

  # plot history
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show()

  # make a prediction
  yhat = model.predict(test_X)
  #print(yhat.shape)
  test_X = test_X[:,-1,:].reshape((test_X.shape[0], test_X.shape[2]))

  # invert scaling for forecast
  yhat = yhat[:,-1,:].reshape((yhat.shape[0], yhat.shape[2]))
  inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
  inv_yhat = sc_y.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  #print(inv_yhat)

  # invert scaling for actual
  y_test_normalized = y_test_normalized.reshape((len(y_test_normalized), 1))
  inv_y = concatenate((Yt_test, test_X[:, 1:]), axis=1)
  inv_y = sc_y.inverse_transform(inv_y)
  inv_y = inv_y[:,0]


  #plt.plot(inv_y, 'go', linewidth=4, label='Actual Value')
  #plt.plot(inv_yhat,'rx--', linewidth=2, label='Predicted Value')
  #plt.legend()
  #plt.savefig("abc.png")
  #files.download("abc.png")
  #plt.show()

  fig2 = plt.figure(figsize=(8, 6)) # The size of the figure is specified as (width, height) in inches

  # lines:
  l1 = fig2.add_subplot(111).plot(inv_y, 'co-')
  l2 = fig2.add_subplot(111).plot(inv_yhat,'rx--')
  fig2.add_subplot(111).legend(['Actual', 'Predicted'], loc=0)


  # axes:
  fig2.add_subplot(111).grid(True)

  # labels:
  fig2.add_subplot(111).set_xlabel(r"Days", fontsize=15)
  fig2.add_subplot(111).set_ylabel(r"Global_active_power (in Kilowatts)", fontsize=15)
  fig2.add_subplot(111).set_title(r"Actual vs. Predicted for daily case", fontsize=15)

  # saving:
  fig2.savefig("lstm_multivariate_days.png", dpi=150)
  files.download("lstm_multivariate_days.png")
  fig2.show()



  #print(inv_y.shape)

  #print(inv_yhat.shape)
  #print(inv_y[:])

  # calculate RMSE
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
  mape = np.mean(np.abs((inv_y.flatten() - inv_yhat.flatten()) /inv_y.flatten())) * 100
  print(f'Mean absolute percentage error is: {mape}')
  mae_error = mae(inv_y.flatten(),inv_yhat.flatten())
  print(f'Mean Absolute Error is: {mae_error} ')

In [ ]:
X = data_response_weekly.copy()
Multivariate_LSTM_timesteps(X,2)

## For months

### For 1 layer

In [ ]:
dataset = weather_response_monthly
#train,test = dataset.values[:-48],dataset.values[-48:]
#dataset[1:-331]
dataset[1:-11]

In [ ]:
def Multivariate_LSTM(X):
  #Splitting the data into train and test sets. Since we are dealing with time-series data we cannot split it into randomized split. It must be a sequential split.
  #Here we are using first 75% data for training and 25% data for testing

  X_train = X.iloc[1:-11,:-1]
  X_test = X.iloc[-11:,:-1]
  y_train = X.iloc[1:-11,-1]
  y_test = X.iloc[-11:,-1]

  y_train = y_train.values.reshape(len(y_train),1)
  y_test = y_test.values.reshape(len(y_test),1)

  ####Scaling the data after train-test split
  sc_X = StandardScaler()
  sc_y = StandardScaler()
  X_train_normalized = sc_X.fit_transform(X_train)
  X_test_normalized = sc_X.transform(X_test)
  y_train_normalized = sc_y.fit_transform(y_train)
  y_test_normalized = sc_y.transform(y_test)
  
  # reshape input to be 3D [samples, timesteps, features]
  train_X = X_train_normalized.reshape((X_train_normalized.shape[0], 1, X_train_normalized.shape[1]))
  test_X = X_test_normalized.reshape((X_test_normalized.shape[0], 1, X_test_normalized.shape[1]))

  n_input = 1
  n_features = X_train_normalized.shape[1]

  # define model
  model = Sequential()
  model.add(LSTM(50, activation='relu', input_shape=(n_input, n_features)))
  model.add(Dropout(0.3))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mse')
  # fit model
  history = model.fit(train_X, y_train_normalized, epochs=100, batch_size=2, validation_data=(test_X, y_test_normalized), 
                     verbose=1, shuffle=False)

  model.summary()

  # plot history
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show()

  # make a prediction
  yhat = model.predict(test_X)
  test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
  # invert scaling for forecast
  inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
  inv_yhat = sc_y.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  print(inv_yhat)
  # invert scaling for actual
  y_test_normalized = y_test_normalized.reshape((len(y_test_normalized), 1))
  inv_y = concatenate((y_test_normalized, test_X[:, 1:]), axis=1)
  inv_y = sc_y.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  print(inv_y)

  
  #plt.plot(inv_y, 'go', linewidth=4, label='Actual Value')
  #plt.plot(inv_yhat,'rx--', linewidth=2, label='Predicted Value')
  #plt.legend()
  #plt.savefig("abc.png")
  #files.download("abc.png")
  #plt.show()

  fig2 = plt.figure(figsize=(8, 6)) # The size of the figure is specified as (width, height) in inches

  # lines:
  l1 = fig2.add_subplot(111).plot(inv_y, 'co-')
  l2 = fig2.add_subplot(111).plot(inv_yhat,'rx--')
  fig2.add_subplot(111).legend(['Actual', 'Predicted'], loc=0)


  # axes:
  fig2.add_subplot(111).grid(True)

  # labels:
  fig2.add_subplot(111).set_xlabel(r"Months", fontsize=15)
  fig2.add_subplot(111).set_ylabel(r"Global_active_power (in Kilowatts)", fontsize=15)
  fig2.add_subplot(111).set_title(r"Actual vs. Predicted for Monthly case", fontsize=15)

  # saving:
  fig2.savefig("lstm_multivariate_monthly.png", dpi=150)
  files.download("lstm_multivariate_monthly.png")
  fig2.show()

  #print(inv_y.shape)

  #print(inv_yhat.shape)
  #print(inv_y[:])

  res = np.vstack((inv_y, inv_yhat))
  res = res.transpose()
  DF = pd.DataFrame(res)
  # save the dataframe as a csv file
  DF.to_csv("results.csv")

  # calculate RMSE
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
  mape = np.mean(np.abs((inv_y.flatten() - inv_yhat.flatten()) /inv_y.flatten())) * 100
  print(f'Mean absolute percentage error is: {mape}')
  mae_error = mae(inv_y.flatten(),inv_yhat.flatten())
  print(f'Mean Absolute Error is: {mae_error} ')

In [ ]:
X = data_response_monthly.copy()
Multivariate_LSTM(X)

### For multi-layer

In [ ]:
def Multivariate_LSTM_2(X):
  #Splitting the data into train and test sets. Since we are dealing with time-series data we cannot split it into randomized split. It must be a sequential split.
  # Here we are using first 3 years data for training and 1 year data for testing
  
  #X_train = X.iloc[:int(X.shape[0]*0.80),:]
  #X_test = X.iloc[int(X.shape[0]*0.80):,:]
  #y_train = X.iloc[:int(X.shape[0]*0.80),-1]
  #y_test = X.iloc[int(X.shape[0]*0.80):,-1]

  X_train = X.iloc[1:-11,:-1]
  X_test = X.iloc[-11:,:-1]
  y_train = X.iloc[1:-11,-1]
  y_test = X.iloc[-11:,-1]

  y_train = y_train.values.reshape(len(y_train),1)
  y_test = y_test.values.reshape(len(y_test),1)

  ####Scaling the data after train-test split
  sc_X = StandardScaler()
  sc_y = StandardScaler()
  X_train_normalized = sc_X.fit_transform(X_train)
  X_test_normalized = sc_X.transform(X_test)
  y_train_normalized = sc_y.fit_transform(y_train)
  y_test_normalized = sc_y.transform(y_test)

  # reshape input to be 3D [samples, timesteps, features]
  train_X = X_train_normalized.reshape((X_train_normalized.shape[0], 1, X_train_normalized.shape[1]))
  test_X = X_test_normalized.reshape((X_test_normalized.shape[0], 1, X_test_normalized.shape[1]))

  n_input = 1
  n_features = X_train_normalized.shape[1]

  # define model
  model = Sequential()
  model.add(LSTM(50, activation='relu', return_sequences = True, input_shape=(n_input, n_features)))
  model.add(Dropout(0.3))
  model.add(LSTM(30, activation='relu',return_sequences = True))
  model.add(Dropout(0.1))
  # can add more layers if required
  model.add(Dense(1))
  opt = keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer=opt, loss='mse')
  # fit model
  history = model.fit(train_X, y_train_normalized, epochs=150, batch_size=2, validation_data=(test_X, y_test_normalized), 
                      verbose=1, shuffle=False)

  model.summary()

  # plot history
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show()

  # make a prediction
  yhat = model.predict(test_X)
  yhat.shape
  test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
  
  # invert scaling for forecast
  yhat = yhat.reshape((yhat.shape[0], yhat.shape[2]))
  inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
  inv_yhat = sc_y.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  #print(inv_yhat)
  
  # invert scaling for actual
  y_test_normalized = y_test_normalized.reshape((len(y_test_normalized), 1))
  inv_y = concatenate((y_test_normalized, test_X[:, 1:]), axis=1)
  inv_y = sc_y.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  #print(inv_y)


  #plt.plot(inv_y, 'go', linewidth=4, label='Actual Value')
  #plt.plot(inv_yhat,'rx--', linewidth=2, label='Predicted Value')
  #plt.legend()
  #plt.savefig("abc.png")
  #files.download("abc.png")
  #plt.show()

  fig2 = plt.figure(figsize=(8, 6)) # The size of the figure is specified as (width, height) in inches

  # lines:
  l1 = fig2.add_subplot(111).plot(inv_y, 'co-')
  l2 = fig2.add_subplot(111).plot(inv_yhat,'rx--')
  fig2.add_subplot(111).legend(['Actual', 'Predicted'], loc=0)


  # axes:
  fig2.add_subplot(111).grid(True)

  # labels:
  fig2.add_subplot(111).set_xlabel(r"Days", fontsize=15)
  fig2.add_subplot(111).set_ylabel(r"Global_active_power (in Kilowatts)", fontsize=15)
  fig2.add_subplot(111).set_title(r"Actual vs. Predicted for daily case", fontsize=15)

  # saving:
  fig2.savefig("lstm_multivariate_days.png", dpi=150)
  files.download("lstm_multivariate_days.png")
  fig2.show()



  #print(inv_y.shape)

  #print(inv_yhat.shape)
  #print(inv_y[:])

  # calculate RMSE
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
  mape = np.mean(np.abs((inv_y.flatten() - inv_yhat.flatten()) /inv_y.flatten())) * 100
  print(f'Mean absolute percentage error is: {mape}')
  mae_error = mae(inv_y.flatten(),inv_yhat.flatten())
  print(f'Mean Absolute Error is: {mae_error} ')

In [ ]:
X = data_response_monthly.copy()
Multivariate_LSTM_2(X)

### For timesteps

In [ ]:
def timesteps(x_train,y_train,lag):
  X = []
  Y = []
  num_samples = x_train.shape[0]

  for i in range(lag, num_samples):

    temp = x_train[i-lag:i, :]
    #X.append(temp.reshape(1,temp.shape[0],temp.shape[1]))
    X.append(temp)
    Y.append(y_train[i])

  X,Y = np.array(X), np.array(Y)
  
  return X,Y

In [ ]:
def Multivariate_LSTM_timesteps(X,lag):
  #Splitting the data into train and test sets. Since we are dealing with time-series data we cannot split it into randomized split. It must be a sequential split.
  #Here we are using first 75% data for training and 25% data for testing
  #X_train = X.iloc[:int(X.shape[0]*0.80),:]
  #X_test = X.iloc[int(X.shape[0]*0.80):,:]
  #y_train = X.iloc[:int(X.shape[0]*0.80),-1]
  #y_test = X.iloc[int(X.shape[0]*0.80):,-1]

  X_train = X.iloc[1:-11,:-1]
  X_test = X.iloc[-11:,:-1]
  y_train = X.iloc[1:-11,-1]
  y_test = X.iloc[-11:,-1]

  y_train = y_train.values.reshape(len(y_train),1)
  y_test = y_test.values.reshape(len(y_test),1)

  ####Scaling the data after train-test split
  sc_X = StandardScaler()
  sc_y = StandardScaler()
  X_train_normalized = sc_X.fit_transform(X_train)
  X_test_normalized = sc_X.transform(X_test)
  y_train_normalized = sc_y.fit_transform(y_train)
  y_test_normalized = sc_y.transform(y_test)

  # Creating data in timesteps
  Xt_train,Yt_train = timesteps(X_train_normalized,y_train_normalized,lag)
  Xt_test,Yt_test = timesteps(X_test_normalized,y_test_normalized,lag)
  
  # reshape input to be 3D [samples, timesteps, features]
  train_X = Xt_train
  test_X = Xt_test

  n_input = lag
  n_features = train_X.shape[2]

  # define model
  model = Sequential()
  model.add(LSTM(100, activation='relu', return_sequences = True, input_shape=(n_input, n_features)))
  model.add(Dropout(0.3))
  model.add(LSTM(35, activation='relu',return_sequences = True))
  model.add(Dropout(0.1))
  model.add(Dense(1))
  opt = keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer=opt, loss='mse')
  # fit model
  history = model.fit(train_X, Yt_train, epochs=60, batch_size=2, validation_data=(test_X, Yt_test), 
                     verbose=1, shuffle=False)

  model.summary()

  # plot history
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show()

  # make a prediction
  yhat = model.predict(test_X)
  #print(yhat.shape)
  test_X = test_X[:,-1,:].reshape((test_X.shape[0], test_X.shape[2]))

  # invert scaling for forecast
  yhat = yhat[:,-1,:].reshape((yhat.shape[0], yhat.shape[2]))
  inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
  inv_yhat = sc_y.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  #print(inv_yhat)

  # invert scaling for actual
  y_test_normalized = y_test_normalized.reshape((len(y_test_normalized), 1))
  inv_y = concatenate((Yt_test, test_X[:, 1:]), axis=1)
  inv_y = sc_y.inverse_transform(inv_y)
  inv_y = inv_y[:,0]


  #plt.plot(inv_y, 'go', linewidth=4, label='Actual Value')
  #plt.plot(inv_yhat,'rx--', linewidth=2, label='Predicted Value')
  #plt.legend()
  #plt.savefig("abc.png")
  #files.download("abc.png")
  #plt.show()

  fig2 = plt.figure(figsize=(8, 6)) # The size of the figure is specified as (width, height) in inches

  # lines:
  l1 = fig2.add_subplot(111).plot(inv_y, 'co-')
  l2 = fig2.add_subplot(111).plot(inv_yhat,'rx--')
  fig2.add_subplot(111).legend(['Actual', 'Predicted'], loc=0)


  # axes:
  fig2.add_subplot(111).grid(True)

  # labels:
  fig2.add_subplot(111).set_xlabel(r"Days", fontsize=15)
  fig2.add_subplot(111).set_ylabel(r"Global_active_power (in Kilowatts)", fontsize=15)
  fig2.add_subplot(111).set_title(r"Actual vs. Predicted for daily case", fontsize=15)

  # saving:
  fig2.savefig("lstm_multivariate_days.png", dpi=150)
  files.download("lstm_multivariate_days.png")
  fig2.show()



  #print(inv_y.shape)

  #print(inv_yhat.shape)
  #print(inv_y[:])

  # calculate RMSE
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
  mape = np.mean(np.abs((inv_y.flatten() - inv_yhat.flatten()) /inv_y.flatten())) * 100
  print(f'Mean absolute percentage error is: {mape}')
  mae_error = mae(inv_y.flatten(),inv_yhat.flatten())
  print(f'Mean Absolute Error is: {mae_error} ')

In [ ]:
X = data_response_monthly.copy()
Multivariate_LSTM_timesteps(X,2)